In [7]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import mplfinance as mpf
import Data_for_bp as dfb
import pickle

In [8]:
def ladderize_open(tick_data, grid_size):
    """
    Convert tick data into step-based data using a specified grid size.

    :param tick_data: A pandas Series of tick data.
    :param grid_size: The size of the grid to discretize the tick data.
    :return: A pandas Series of ladderized data.
    """
    ladderized_data = [tick_data.iloc[0]]
    for i in range(1, len(tick_data)):
        if tick_data.iloc[i] > ladderized_data[-1] + grid_size:
            ladderized_data.append(ladderized_data[-1] + grid_size)
        elif tick_data.iloc[i] < ladderized_data[-1] - grid_size:
            ladderized_data.append(ladderized_data[-1] - grid_size)
        else:
            ladderized_data.append(ladderized_data[-1])
    # Adding the final close price
    ladderized_data[-1]=tick_data.iloc[-1]
    return pd.Series(ladderized_data, index=tick_data.index)
def ladderize_absolute(tick_data, grid_size):
    """
    Convert tick data into step-based data using a specified grid size.

    :param tick_data: A pandas Series of tick data.
    :param grid_size: The size of the grid to discretize the tick data.
    :return: A pandas Series of ladderized data.
    """
    # Initialize ladder at the nearest rounded price level based on grid size
    ladderized_data = [(tick_data.iloc[0] / grid_size).round() * grid_size]
    for i in range(1, len(tick_data)):
        current_tick = tick_data.iloc[i]
        last_ladder_level = ladderized_data[-1]
        if current_tick > last_ladder_level + grid_size:
            ladderized_data.append(last_ladder_level + grid_size)
        elif current_tick < last_ladder_level - grid_size:
            ladderized_data.append(last_ladder_level - grid_size)
        else:
            ladderized_data.append(last_ladder_level)
    # Adding the final close price
    ladderized_data[-1] = tick_data.iloc[-1]
    return pd.Series(ladderized_data, index=tick_data.index)

In [9]:
def count_ticks_by_date(tick_data):
    # Group the data by date
    grouped = tick_data.groupby(tick_data.index.date)
    
    # Initialize an empty array to hold the count of ticks for each date
    D = len(grouped)
    T = np.zeros(D)
    
    # Initialize an index variable for populating the T array
    T_index = 0
    
    # Loop through each group and count the number of ticks
    for date, group in grouped:
        T[T_index] = len(group)
        print(f"Date: {date}, Number of ticks: {len(group)}")
        T_index += 1
    
    # Convert the array to integers
    T = T.astype(int)
    return T,D

In [26]:
def B_value(tick_data, grid_size, ladderized_function):
    '''
    Convert tick data to binomial series and also return the original tick data values
    corresponding to the indices where the binomial series exists.
    '''
    ladderized_data = ladderized_function(tick_data, grid_size)
    jumps = dfb.filter_jumps(ladderized_data)
    aggregated_diff = dfb.aggregate_differences(jumps)
    binomial_data = aggregated_diff.diff()
    
    # Drop NaNs from binomial_data and get the corresponding original tick_data values
    binomial_data.dropna(inplace=True)
    original_values_at_binomial_index = tick_data[binomial_data.index]
    
    return binomial_data.to_list(), original_values_at_binomial_index.tolist()


In [25]:
def compute_pnl_t(G,T,n,tick_data,d,position):
    #initialize pnl matrix for day d, T[d] period
    B, ladder_data = B_value(tick_data, G, ladderize_open)
    pnl = np.zeros(T[d])
    for t in range (T[d]-1):
        if B[t] > 0: ################ how to change t when day change>> Can i use if else for T period?
            position-=n
        elif B[t] < 0: ################ lets try one day first this can neglect for now
            position+=n
        else:
            position+=0
        pnl[t+1] = position * G * B[t] + pnl[t]
    pnl_T= pnl[-1] + position*(tick_data.iloc[T[d]-1]-G)
    return pnl,pnl_T,position 

In [22]:
def net_profit(G,T,n,tick_data):
    profit = np.zeros(D)
    for d in range (D):
        if d == 0:
            position = 0
            pnl, pnl_T, position = compute_pnl_t(G, T, n, tick_data,d,position)
            profit[d] = pnl_T
        else:
            pnl, pnl_T, position = compute_pnl_t(G, T, n, tick_data,d,position)
            profit[d] = pnl_T + profit[d-1]
    net_profit = profit[-1] 
    return -net_profit

In [23]:
def position_constraint(params, T, n, tick_data,d,position):
    G = params[0]
    _, _, position = compute_pnl_t(G, T, n, tick_data,d,position)  # You might want to update these arguments based on your loop in net_profit
    return min(10000000 - abs(position), abs(position) + 10000000)

def pnl_constraint(params, T, n, tick_data,d,position):
    G = params[0]
    pnl, _, _= compute_pnl_t(G, T, n, tick_data,d,position)  # You might want to update these arguments based on your loop in net_profit
    return min(pnl) + 150000

def profit_constraint(params, T, n, tick_data):
    G = params[0]
    net_profit_value = net_profit(G, T, n, tick_data)
    return net_profit_value + 500000


In [12]:
#fix n constant for now
n = 1000 #fix n
# lets try data of one day
tick_data = dfb.get_tick_data('15 Aug 2023','16 Aug 2023')['EURUSD.mid']
T,D =count_ticks_by_date(tick_data)

2023-09-16 18:14:37,400 - findatapy.market.datavendorweb - INFO - Request Dukascopy data
2023-09-16 18:14:37,401 - findatapy.market.datavendorweb - INFO - About to download from Dukascopy... for EURUSD
2023-09-16 18:14:37,473 - findatapy.market.datavendorweb - INFO - Downloading... 2023-08-15 00:00:00 https://www.dukascopy.com/datafeed/EURUSD/2023/07/15/00h_ticks.bi5
2023-09-16 18:14:43,529 - findatapy.market.datavendorweb - INFO - Completed request from Dukascopy
2023-09-16 18:14:45,636 - findatapy.market.ioengine - WARNING - Couldn't push MarketDataRequest_858__abstract_curve_key-None__base_depos_currencies-EUR_GBP_AUD_NZD_USD_CAD_CHF_NOK_SEK_JPY__base_depos_tenor-ON_TN_SN_1W_2W_3W_1M_... to Redis: Error 10061 connecting to 127.0.0.1:6379. No connection could be made because the target machine actively refused it.
Date: 2023-08-15, Number of ticks: 108011


In [23]:
B,V = B_value(tick_data, 0.001, ladderize_open)
len(B)

24

In [27]:
V

[1.0907800197601318,
 1.091825008392334,
 1.0927850008010864,
 1.0917699337005615,
 1.0927850008010864,
 1.0937800407409668,
 1.0927749872207642,
 1.0916650295257568,
 1.0907750129699707,
 1.0897600650787354,
 1.0908050537109375,
 1.091784954071045,
 1.0927900075912476,
 1.0917799472808838,
 1.0927900075912476,
 1.093785047531128,
 1.0948200225830078,
 1.0937700271606445,
 1.092764973640442,
 1.0937800407409668,
 1.0927749872207642,
 1.0917799472808838,
 1.0907050371170044,
 1.0903899669647217]

In [ ]:
def c(tick_data,grid_size):
    ladderized_data = ladderized_function(tick_data,grid_size)
    jumps = filter_jumps(ladderized_data)
    binomial_data.dropna(inplace=True)
    return binomial_data.to_list()

In [26]:
#optimixation
constraints = [{'type': 'ineq', 'fun': lambda params: position_constraint(params, T, n, tick_data,d=0,position=0)},
               {'type': 'ineq', 'fun': lambda params: pnl_constraint(params, T, n, tick_data,d=0,position=0)},
               {'type': 'ineq', 'fun': lambda params: profit_constraint(params, T, n, tick_data)}]


# Initial guess for G
initial_G = [0.001]

result = minimize(lambda x: -net_profit(x[0], T, n, tick_data),
                  initial_G,
                  constraints=constraints,
                  bounds=[(0, None)])


# The optimal G should now be stored in result.x
optimal_G = result.x[0]
print(optimal_G)

IndexError: list index out of range